<a href="https://colab.research.google.com/github/dAn-solution/competition/blob/main/Prob_kiva_024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kiva／クラウドファンディングの資金調達額予測
- Lasso回帰 を実施

### Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'

Mounted at /content/drive
/content/drive/My Drive


### ライブラリのインストール、インポート

In [ ]:
# カレントディレクトリを変更
import os
os.chdir('/content/drive/My Drive/Probdata/kiva/')
print(os.getcwd())

/content/drive/My Drive/Probdata/kiva


In [ ]:
class Config():
    root_path = './'
    input_path = os.path.join(root_path, 'input')
    output_path = os.path.join(root_path, 'output')
    result_path = os.path.join(root_path, 'result')
    bert_model_name = 'bert-base-uncased'
    seed = 42
    debug = False

In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings

import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Lasso

### データの読み込み

In [ ]:
train_df = pd.read_csv(f'{Config.input_path}/train.csv')
test_df = pd.read_csv(f'{Config.input_path}/test.csv')
sample_df = pd.read_csv(f'{Config.input_path}/sample_submission.csv')

train_011_df = pd.read_csv(f'{Config.result_path}/train_011.csv')
test_011_df = pd.read_csv(f'{Config.result_path}/test_011.csv')
train_012_df = pd.read_csv(f'{Config.result_path}/train_012.csv')
test_012_df = pd.read_csv(f'{Config.result_path}/test_012.csv')
train_013_df = pd.read_csv(f'{Config.result_path}/train_013.csv')
test_013_df = pd.read_csv(f'{Config.result_path}/test_013.csv')
train_014_df = pd.read_csv(f'{Config.result_path}/train_014.csv')
test_014_df = pd.read_csv(f'{Config.result_path}/test_014.csv')
train_015_df = pd.read_csv(f'{Config.result_path}/train_015.csv')
test_015_df = pd.read_csv(f'{Config.result_path}/test_015.csv')
train_016_df = pd.read_csv(f'{Config.result_path}/train_016.csv')
test_016_df = pd.read_csv(f'{Config.result_path}/test_016.csv')
train_020_df = pd.read_csv(f'{Config.result_path}/train_020.csv')
test_020_df = pd.read_csv(f'{Config.result_path}/test_020.csv')

## Lasso回帰の実行

In [ ]:
target_column = "LOAN_AMOUNT"
y = train_df[target_column]

In [ ]:
# 合体
X = pd.concat([train_011_df, train_012_df, train_013_df, train_014_df, train_015_df, train_016_df, train_020_df], axis=1)
test_X = pd.concat([test_011_df, test_012_df, test_013_df, test_014_df, test_015_df, test_016_df, test_020_df], axis=1)

In [ ]:
# trainデータを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = Config.seed)

In [ ]:
# 学習/予測

y_oof = np.zeros(len(y_train))
meta_preds = []
y_preds = []
kf = StratifiedKFold(n_splits=5)
for fold, (tr_idx, vl_idx) in enumerate(kf.split(X_train, y_train)):
    x_tr_fold = X.iloc[tr_idx]
    y_tr_fold = y.iloc[tr_idx]
    x_vl_fold = X.iloc[vl_idx]
    y_vl_fold = y.iloc[vl_idx]

    model = Lasso(alpha=0.05)
    model.fit(x_tr_fold, y_tr_fold)
    y_oof[vl_idx] = model.predict(x_vl_fold)
    meta_preds.append(model.predict(X))
    y_preds.append(model.predict(test_X))
    print(
        f'fold {fold} score:', mean_absolute_error(y_oof[vl_idx], y_vl_fold)
    )

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.746e+09, tolerance: 5.955e+06
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


fold 0 score: 147.36827153294564


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.739e+09, tolerance: 6.017e+06
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


fold 1 score: 144.7463006628848


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.714e+09, tolerance: 6.132e+06
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


fold 2 score: 145.3370772042651


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.747e+09, tolerance: 6.024e+06
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


fold 3 score: 144.60904244131328
fold 4 score: 144.62543784696317


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.759e+09, tolerance: 5.893e+06
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


In [ ]:
# メタモデルへの引き渡し
train_024_df = pd.DataFrame(np.mean(meta_preds, axis=0))
train_024_df.to_csv(os.path.join(Config.result_path, "train_024.csv"), index=False)


test_024_df = pd.DataFrame(np.mean(y_preds, axis=0))
test_024_df.to_csv(os.path.join(Config.result_path, "test_024.csv"), index=False)

### 提出データの作成

In [ ]:
sample_df["LOAN_AMOUNT"] = np.mean(y_preds, axis=0)
print(sample_df)

       LOAN_ID  LOAN_AMOUNT
0      2041445   466.693036
1      1944435   655.516795
2      2083354   360.133628
3      1993565   915.668570
4      2064272   350.426204
...        ...          ...
91817  1993862   255.386088
91818  2015070   221.775939
91819  1950349   547.402762
91820  1921580  1531.608342
91821  1976733   560.440221

[91822 rows x 2 columns]


In [ ]:
# submitファイルのファイル名に利用する作成時刻の取得

now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=1)))
date_time = "{0:%Y%m%d_%H%M}".format(now)

In [ ]:
# submitファイルの出力(Google Driveに出力）
# 形式：submit_yyyymmdd_hhmm.csv

sample_df.to_csv(os.path.join(Config.output_path, f"submit_{date_time}.csv"), index=False)


In [ ]:
test_024_df.describe()

0
count  91822.000000
mean     679.325537
std      645.944499
min     -784.912687
25%      350.182354
50%      489.379788
75%      770.903927
max     9922.688018